In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("dongguk_update.pdf")
docs=loader.load()
print(len(docs))

/Users/lee/.pyenv/versions/chatDPT/lib/python3.10/site-packages/langchain_community/document_loaders/parsers/pdf.py:322: UserWarning: Warning: Empty content on page 0 of document dongguk_update.pdf
  warnings.warn(


240


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
split_documents = text_splitter.split_documents(docs)

In [8]:
from langchain_upstage import UpstageEmbeddings
embedding = UpstageEmbeddings(model="solar-embedding-1-large-passage")

In [10]:
import os

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name='dongguk'
pinecone_api_key=os.environ.get("PINECONE_API_KEY")
pc=Pinecone(api_key=pinecone_api_key)

database = PineconeVectorStore.from_documents(split_documents, embedding, index_name=index_name)

In [31]:
query = '1학기 수강신청 기간 알려줘'
retrieved_docs = database.similarity_search(query)

In [32]:
retrieved_docs

[Document(id='894e8a94-3435-469b-9313-f50be7ba4019', metadata={'author': 'Administrator', 'creationDate': "D:20240929115640+09'00'", 'creator': 'Hwp 2018 10.0.0.13015', 'file_path': 'dongguk_update.pdf', 'format': 'PDF 1.4', 'keywords': '', 'modDate': "D:20240929115640+09'00'", 'page': 18.0, 'producer': 'Hancom PDF 1.3.0.542', 'source': 'dongguk_update.pdf', 'subject': '', 'title': '', 'total_pages': 240.0, 'trapped': ''}, page_content='17\nⅡ. 수강신청 및 수업관련제도▶▶\n2. 희망강의신청 \n가. 취지\n  (1) 수강신청 대비 ‘쇼핑카트’ 개념으로서, 수강신청 편의 및 혜택 제공\n  (2) 신속하고 안정적인 수강신청 시스템 운영 \n  (3) 강좌별 수강 수요현황을 파악하여 강좌 분반 및 강의실 조정에 활용  \n나. 대상 및 신청기간 \n다. 신청방법 : mDRIMS → 학사정보 → 교과수업 → 수강신청관리 → 희망강의신청\n라. 신청가능학점 : 수강을 희망하는 강좌 24학점 이내로 신청 가능\n                 ※해당학기(희망강의 신청 시점 학기) 지도교수 상담 학생은 27학점까지 신청 가능\n마. 유의사항 \n  (1) 희망강의신청 결과와 관계없이 반드시 본 수강신청 기간에 수강신청을 완료해야 하며, 쇼핑\n     카트 클릭으로 수강신청을 신속하고 편리하게 할수 있음.\n  (2) 희망강의는 중복(시간, 취득과목) 및 제한사항(학년, 학과, 인원)에 상관없이 신청가능하지만 \n     수강신청기간에는 신청이 불가할 수 있으므로 유의하기 바람.\n  (3) Dream PATH 핵심역량진단자

In [33]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [34]:
from langchain_core.prompts import PromptTemplate

# 프롬프트 생성
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    You are an aiassistant who answers questions related to school life of Dongguk University students.
    Use the following pieces of retrieved context to answer the question. 
    Please answer in the questioning language. For example, if the Question is in English, answer in English, and answer in Chinese
    Please present the page of the document you referred to when answering in the following format.
    If a user's question varies depending on various situations or environments, encourages the user to ask more specific questions, please.
    You should never answer inaccurate or incorrect content. If you can't answer the content, don't provide a page. let's think step by step
    # format : 이 답변은 2024 신입생 학업이수가이드 ??? 페이지를 참고해 작성되었습니다.

    Retrieved Context: {context}

    Question: {question}

    Answer:
    """
)

In [35]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [36]:
ai_message = qa_chain.invoke({'query': query})

In [37]:
ai_message['result']

'2024학년도 1학기 수강신청 기간은 다음과 같습니다:\n\n- 3~6학년: 2024년 2월 2일(금) 10:00부터 23:59까지\n- 1~2학년: 2024년 2월 5일(월) 10:00부터 23:59까지\n- 다전공자(복수/연계/융합/학생설계전공): 2024년 2월 6일(화) 10:00부터 23:59까지\n- 전체 학생: 2024년 2월 7일(수) 10:00부터 2월 8일(목) 17:00까지\n\n이 답변은 2024 신입생 학업이수가이드 16 페이지를 참고해 작성되었습니다.'